# FAKE NEWS CLASSIFIER USING LSTM

IN THIS NOTEBOOK WE WILL CLASSIFY WHETHER THE NEWS IS FAKE OR NOT USING LSTM MODEL.

1--> REAL

0--> FAKE

WE WILL PERFORM SOME TEXT PREPROCESSING( STEMMING, REMOVAL OF STOP WORDS, CONVERTING TEXT INTO VECTORS)

THEN WE WILL BUILD OUR LSTM MODEL TO CLASSIFY THE NEWS

In [2]:
# IMPORTING LIBRARY
import pandas as pd

In [5]:
from google.colab import files
files.upload()


Saving My_compressed1.csv to My_compressed1.csv


In [6]:
# OUR DATA FRAME
df= pd.read_csv('My_compressed1.csv')
df.head()

,content,type
0,As if there was not enough to do to prepare fo...,reliable
1,"World News | Tue Nov 22, 2016 | 11:54pm IST Ne...",reliable
2,"WARNER--Robert, has been the single most influ...",reliable
3,"Mr. Flynn added, ''What we have to refrain fro...",reliable
4,“Whatever this miraculous thing is that you’re...,reliable


In [7]:
df = df[["content", 'type']]
df

,content,type
0,As if there was not enough to do to prepare fo...,reliable
1,"World News | Tue Nov 22, 2016 | 11:54pm IST Ne...",reliable
2,"WARNER--Robert, has been the single most influ...",reliable
3,"Mr. Flynn added, ''What we have to refrain fro...",reliable
4,“Whatever this miraculous thing is that you’re...,reliable
...,...,...
19713,X22Report Central Banks Are Ready to Make a Mo...,fake
19714,This Congressional Hearing Will Blow You Away!...,fake
19715,Former Enemies Japan and Russia to Trade LNG?\...,fake
19716,he Shocking Truth About Wall Street Stock Reco...,fake


In [ ]:
df['type'].value_counts()

In [10]:
df_sample = df

In [ ]:
# df_sample = df.sample(frac =0.1)


In [11]:
df_sample.shape

(19718, 2)

In [ ]:
df_sample.type.value_counts()
df_sample

In [12]:
df_0 = df_sample[df_sample['type'] == 'reliable']
df_0.shape

(9549, 2)

In [13]:
df_1 = df_sample[df_sample['type'] == 'bias']
print(df_1.shape)
df_2 = df_sample[df_sample['type'] == 'fake']
print(df_2.shape)
df3 = pd.concat([df_1, df_2], axis=0)
df3 = df3.reset_index()
df3 = df3.copy()
df3

(5772, 2)
(4397, 2)


,index,content,type
0,9549,Mark Steyn to Tucker Carlson: Canadians Can Be...,bias
1,9550,FAIR USE NOTICE. Many of the stories on this s...,bias
2,9551,(CNSNews.com) – The president’s budget require...,bias
3,9552,During an appearance before the U.K.’s House o...,bias
4,9553,Ice collected from Greenland will be displayed...,bias
...,...,...,...
10164,19713,X22Report Central Banks Are Ready to Make a Mo...,fake
10165,19714,This Congressional Hearing Will Blow You Away!...,fake
10166,19715,Former Enemies Japan and Russia to Trade LNG?\...,fake
10167,19716,he Shocking Truth About Wall Street Stock Reco...,fake


In [14]:
df_new = pd.concat([df_0, df3] ,axis=0)
df_new.drop(columns=df_new.columns[-1], 
        axis=1, 
        inplace=True)
df_new.shape

(19718, 2)

In [15]:
print(df_0.shape)
print(df_1.shape)
print(df_2.shape)
print(df_new.shape)

(9549, 2)
(5772, 2)
(4397, 2)
(19718, 2)


In [17]:
df = df_new
df.shape

(19718, 2)

In [18]:
df.type.value_counts()

reliable    9549
bias        5772
fake        4397
Name: type, dtype: int64

In [19]:
df.type = df.type.replace("bias", "fake")
df.type.value_counts()

fake        10169
reliable     9549
Name: type, dtype: int64

In [55]:
df.type = df.type.replace('fake', 0).replace('reliable',1)

In [56]:
df.type

0        1
1        1
2        1
3        1
4        1
        ..
10164    0
10165    0
10166    0
10167    0
10168    0
Name: type, Length: 19718, dtype: int64

In [21]:
max_size = 50
df['content'] = df['content'].apply(lambda x: ' '.join(x.split(maxsplit=max_size)[:max_size]))
df['content'][12]

12    Goldman now faces a choice between two distinc...
12    E-mail: Screen Name: Password: Confirm passwor...
Name: content, dtype: object

In [22]:
## GETTING INDEPENDENT FEATURES
x= df.drop('type',axis=1)

In [57]:
# DEPENDENT FEATURE OR TARGET FEATURE
y=df['type']

In [24]:
# LOOKING AT THE SHAPE OF OUR IINDEPENDENT FEATURES DATASET
x.shape

(19718, 1)

In [60]:
# LOOKING AT THE SHAPE OF OUR DEPENDENT FEATURE
y.shape

(19718,)

In [ ]:
y

In [26]:
# IMPORTING MORE NECESSARY LIBRARIES
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [27]:
# vocabularry size
voc_size=4000

# NATURAL LANGUAGE PROCESSING

In [28]:
# COPYING OUR INDEPENDENT FEATURE DATASET 'x' INTO NEW VARIABLE 'messages'
messages= x.copy()

In [29]:
# RESETTING THE INDEX VALUES
messages.reset_index(inplace=True)

In [30]:
# IMPORTING LIBRARIES FOR NATURAL LANGUAGE PROCESSING
import nltk
import re
from nltk.corpus import stopwords

In [31]:
# DOWNLOADING THE STOPWORDS
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
 messages['content']

0        As if there was not enough to do to prepare fo...
1        World News | Tue Nov 22, 2016 | 11:54pm IST Ne...
2        WARNER--Robert, has been the single most influ...
3        Mr. Flynn added, ''What we have to refrain fro...
4        “Whatever this miraculous thing is that you’re...
                               ...                        
19713    X22Report Central Banks Are Ready to Make a Mo...
19714    This Congressional Hearing Will Blow You Away!...
19715    Former Enemies Japan and Russia to Trade LNG? ...
19716    he Shocking Truth About Wall Street Stock Reco...
19717    .Law Domains Won’t Boost Your Website’s SEO (B...
Name: content, Length: 19718, dtype: object

In [33]:
# TEXT PREPROCESSING--> STEMMING, REMOVAL OF STOP WORDS, CONVERTING INTO LOWER CASE
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['content'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [35]:
# LOOKING AT OUR FINAL CORPUS
corpus[2]

'warner robert singl influenc busi person life irreplac life forev memori larri nagl'

# ONE HOT REPRESENTATION

In [36]:
from tensorflow.keras.preprocessing.text import one_hot

In [37]:
onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr[0])

[791, 2440, 3028, 1987, 1866, 1293, 2284, 1061, 1055, 225, 1984, 420, 1086, 791, 3665, 299, 225, 2186, 2473, 1450, 1102, 28]


# EMBEDDING REPRESENTATION

In [38]:
# SETTING OUR SENTENCE LENGTH = 20
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1450 1102   28]
 [   0    0    0 ... 3846 1221 3062]
 [   0    0    0 ... 1648 3834 3628]
 ...
 [   0    0    0 ... 3943 2133  105]
 [   0    0    0 ... 1938 1689  314]
 [   0    0    0 ... 3447 3431 1381]]


In [ ]:
len(embedded_docs)

# CREATING OUR LSTM MODEL

In [39]:
## CREATING OUR LSTM MODEL
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 40)            160000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 216,501
Trainable params: 216,501
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
# CONVERTING X AND Y INTO ARRAYS
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)
x_final.shape, y_final.shape

((19718, 50), (19718,))

In [62]:
y_final

array([1, 1, 1, ..., 0, 0, 0])

In [63]:
# SPLITTING THE DATA INTO TRAINING AND TEST SETS
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [66]:
y_train.shape

(13211,)

# MODEL TRAINING

In [67]:
# FITTING NTO THE MODEL
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64, verbose = True)

Epoch 1/10
207/207 [==============================] - 11s 23ms/step - loss: 0.4514 - accuracy: 0.7729 - val_loss: 0.3676 - val_accuracy: 0.8460
Epoch 2/10
207/207 [==============================] - 4s 20ms/step - loss: 0.2701 - accuracy: 0.8912 - val_loss: 0.3217 - val_accuracy: 0.8658
Epoch 3/10
207/207 [==============================] - 4s 20ms/step - loss: 0.2149 - accuracy: 0.9154 - val_loss: 0.3344 - val_accuracy: 0.8621
Epoch 4/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1725 - accuracy: 0.9372 - val_loss: 0.3703 - val_accuracy: 0.8578
Epoch 5/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1300 - accuracy: 0.9549 - val_loss: 0.4564 - val_accuracy: 0.8445
Epoch 6/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1074 - accuracy: 0.9637 - val_loss: 0.4675 - val_accuracy: 0.8568
Epoch 7/10
207/207 [==============================] - 4s 20ms/step - loss: 0.0805 - accuracy: 0.9738 - val_loss: 0.5768 - val_accuracy: 0.852

In [69]:
# OUR PREDICTION VARIABLE
y_pred= model.predict(x_test)

In [70]:
y_pred

array([[4.4833374e-04],
       [9.8280209e-01],
       [9.9990284e-01],
       ...,
       [1.2782029e-03],
       [5.8743805e-01],
       [9.3981478e-05]], dtype=float32)

In [72]:
y_pred = [int(i > .5) for i in y_pred]

In [74]:
y_pred[2]

1

In [75]:
# IMPORTING LIBRARIES TO SEE THE ACCURACY
from sklearn.metrics import confusion_matrix, accuracy_score

# PRINTING CONFUSION MATRIX
cm= confusion_matrix(y_test,y_pred)
print(cm)

[[2832  535]
 [ 455 2685]]


In [76]:
# ACCURACY SCORE
ac= accuracy_score(y_test,y_pred)
print(ac)

0.8478561549100968


##### ACCURACY SCORE= 91.66 %

# ADDING DROPOUT LAYER

In [77]:
from tensorflow.keras.layers import Dropout


In [78]:
## CREATING MODEL WITH DROPOUT LAYER
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(Dropout(0.3))
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 40)            160000    
                                                                 
 dropout (Dropout)           (None, 50, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,501
Trainable params: 216,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# CONVERTING X AND Y VARIABLES INTO ARRAYS
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [79]:
# FITTING INTO THE MODEL
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
207/207 [==============================] - 7s 22ms/step - loss: 0.4765 - accuracy: 0.7563 - val_loss: 0.3244 - val_accuracy: 0.8606
Epoch 2/10
207/207 [==============================] - 4s 20ms/step - loss: 0.2764 - accuracy: 0.8886 - val_loss: 0.3218 - val_accuracy: 0.8703
Epoch 3/10
207/207 [==============================] - 4s 20ms/step - loss: 0.2260 - accuracy: 0.9126 - val_loss: 0.3242 - val_accuracy: 0.8611
Epoch 4/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1870 - accuracy: 0.9277 - val_loss: 0.3681 - val_accuracy: 0.8625
Epoch 5/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1547 - accuracy: 0.9440 - val_loss: 0.4117 - val_accuracy: 0.8506
Epoch 6/10
207/207 [==============================] - 4s 19ms/step - loss: 0.1269 - accuracy: 0.9540 - val_loss: 0.4545 - val_accuracy: 0.8598
Epoch 7/10
207/207 [==============================] - 4s 20ms/step - loss: 0.1129 - accuracy: 0.9601 - val_loss: 0.4669 - val_accuracy: 0.8523

In [82]:
# PREDICTION MODEL
y_pred= model.predict(x_test)
y_pred = [int(i > .5) for i in y_pred]

In [83]:
from sklearn.metrics import confusion_matrix, accuracy_score

# PRINTING CONFUSION MATRIX
cmm= confusion_matrix(y_test,y_pred)
print(cmm)

[[2915  452]
 [ 547 2593]]


In [84]:
# ACCURACY SCORE
ac= accuracy_score(y_test,y_pred)
print(ac)

0.8464730290456431


In [86]:
x_test[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3730,  198, 1385, 3012, 1866, 3293,  633,
       1558, 3927, 2366,  756, 2718,  650, 2802, 2284, 3293, 1416,  184,
       2729, 1165,  536,  564, 3293,  955], dtype=int32)

In [90]:
y_pred_= model.predict(x_test[[3]])
y_pred_ = [int(i > .5) for i in y_pred]

In [91]:
y_pred_

[0]

In [94]:
x_pp = "According to a report by the news agency Kiev Independent, the Ukrainian armed forces have said that over 12,000 Russian troops have been killed. They have also said that over 1,105 armoured personnel carriers, 526 vehicles and 60 fuel tanks have also been destroyed."

In [96]:
input = [x_pp]

In [ ]:
i

In [97]:
# TEXT PREPROCESSING--> STEMMING, REMOVAL OF STOP WORDS, CONVERTING INTO LOWER CASE
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(input)):
    review = re.sub('[^a-zA-Z]', ' ',input)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr[0]) 

sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

TypeError: ignored